In [5]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
import json
import random

In [ ]:
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
PATH_TO_PATHS_AND_LABELS = "artifacts/paths_labels.json"

In [ ]:
with open(PATH_TO_PATHS_AND_LABELS, "r") as f:
    data = json.load(f)

paths = data["paths"]
labels = data["labels"]
labels_for_subclasses = data["labels_for_subclasses"]


Trenutno ne garantiram da ce podklase biti tocno raspoređene po datasetovima. To možda nije bitno za normalni model.

In [ ]:
from sklearn.model_selection import train_test_split

train_val_paths, test_paths, train_val_labels, test_labels = train_test_split(
    paths, 
    labels, 
    test_size=0.2, 
    random_state=42, 
    shuffle=True,
    stratify=labels  # Keeps class distribution balanced
)

train_paths, val_paths, train_labels, val_labels = train_test_split(
    train_val_paths, 
    train_val_labels, 
    test_size=0.25, 
    random_state=42, 
    shuffle=True,
    stratify=train_val_labels
)

print(f"Training samples: {len(train_paths)}")
print(f"Validation samples: {len(val_paths)}")
print(f"Testing samples: {len(test_paths)}")

Training samples: 10182
Validation samples: 3394
Testing samples: 3394


In [ ]:
def load_and_preprocess_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    #img = tf.image.rgb_to_grayscale(img)
    img = tf.image.resize(img, (512,512))
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

def create_dataset(paths, labels, is_for_training = False):
    if is_for_training:
        dataset = dataset.shuffle(buffer_size=len(paths))
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(32).prefetch(tf.data.AUTOTUNE)
    return ds

In [ ]:
train_ds = create_dataset(train_paths, train_labels, is_for_training=True)
val_ds = create_dataset(val_paths, val_labels)
test_ds = create_dataset(test_paths, test_labels)